In [1]:
from datasets import load_dataset
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import display, Markdown, Latex
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.set_grad_enabled(False)

In [2]:
math500 = load_dataset("HuggingFaceH4/MATH-500")

device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B").to(device)
model.eval()


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [9]:
def extract_reasoning_steps(response):
    """
    Extract the chain of thought steps and final answer from the model response.
    
    Args:
        response (str): The model's response containing reasoning steps and answer
        
    Returns:
        tuple: (list of reasoning steps, final answer)
    """
    # Check if the response contains the "</think>" tag
    if "</think>" in response:
        # Split the response into reasoning part and answer part
        reasoning_part, final_answer = response.split("</think>", 1)
    else:
        reasoning_part = response
        final_answer = None
        
    # Split the reasoning part into individual steps
    # Steps are separated by double newlines
    reasoning_steps = reasoning_part.split("\n\n")
                
    return reasoning_steps, final_answer

def process_example(example):
    problem = example["problem"]
    messages = [
        {"role": "user", "content": problem}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=4096
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    steps, model_answer = extract_reasoning_steps(response)

    return {
        "prompt": problem,
        "steps": steps,
        "answer": example["answer"],
        "model_answer": model_answer,
    }


s = process_example(math500["test"][0])

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [15]:
newset = math500["test"].select(range(3))
newset = newset.map(process_example)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [16]:
newset.save_to_disk("math500_test.hf")

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]